<a href="https://colab.research.google.com/github/vardhanreddy2003/Deep_learning/blob/main/keras_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("/content/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()["Outcome"]

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler=StandardScaler()

In [8]:
X=scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
import tensorflow
from tensorflow import keras
from keras.layers import Dense
from keras import Sequential

In [12]:
model=Sequential()

model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [14]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=32)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7179 - loss: 0.5887 - val_accuracy: 0.7792 - val_loss: 0.5503
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7258 - loss: 0.5548 - val_accuracy: 0.7792 - val_loss: 0.5255
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7612 - loss: 0.5238 - val_accuracy: 0.7922 - val_loss: 0.5101
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7612 - loss: 0.5022 - val_accuracy: 0.7857 - val_loss: 0.5007
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7694 - loss: 0.4912 - val_accuracy: 0.7857 - val_loss: 0.4945
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7579 - loss: 0.4984 - val_accuracy: 0.7792 - val_loss: 0.4916
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7532 - loss: 0.5008 - val_accuracy: 0.7857 - val_loss: 0.4903
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7549 - loss: 0.4729 - val_accuracy: 0.7857 - 

#1.how to select appropriate optimizer
#2.no of nodes in a layer
#3.how to select no of layers
#4.All in all one model

In [15]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [16]:
import kerastuner as kt

<ipython-input-16-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [17]:
from keras.layers import Input

In [18]:


def build_model(hp):

    model = Sequential()

    # Use Input layer instead of input_dim
    model.add(Input(shape=(8,)))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    # Hyperparameter tuning for optimizer
    optimizer = hp.Choice("optimizer", values=["adam", "sgd", "rmsprop", "adadelta"])

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    return model


In [19]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5)

In [20]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 06s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 18s


In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [22]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7468 - loss: 0.5029 - val_accuracy: 0.7727 - val_loss: 0.5122
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7890 - loss: 0.4672 - val_accuracy: 0.7727 - val_loss: 0.5071
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7774 - loss: 0.4620 - val_accuracy: 0.7792 - val_loss: 0.5022
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7718 - loss: 0.4498 - val_accuracy: 0.7727 - val_loss: 0.5019
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7962 - loss: 0.4335 - val_accuracy: 0.7727 - val_loss: 0.5003
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7934 - loss: 0.4550 - val_accuracy: 0.7727 - val_loss: 0.4993
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7699 - loss: 0.4529 - val_accuracy: 0.7727 - val_loss: 0.4993
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7909 - loss: 0.4387 - val_accuracy: 0.77

In [25]:
def build_model(hp):

  model=Sequential()

  units=hp.Int("units",8,128)

  model.add(Input(shape=(8,)))
  model.add(Dense(units=units,activation="relu"))


  model.add(Dense(1,activation="sigmoid"))


  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [26]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [27]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [28]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [29]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: adam
Score: 0.7727272510528564

Trial 3 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7207792401313782

Trial 1 summary
Hyperparameters:
optimizer: sgd
Score: 0.6948052048683167

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.48051947355270386


In [30]:
def build_model(hp):

  model=Sequential()

  model.add(Dense(72,activation="relu",input_dim="8"))
  for i in range(hp.Int("num_layers",1,10)):

    model.add(Dense(72,activation="relu"))

  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model


In [31]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [32]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [33]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [35]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: adam
Score: 0.7727272510528564

Trial 3 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7207792401313782

Trial 1 summary
Hyperparameters:
optimizer: sgd
Score: 0.6948052048683167

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.48051947355270386


In [64]:
from keras.layers import Dropout

In [71]:
def build_model(hp):

  model=Sequential()
  counter=0

  units=hp.Int("units",8,128)
  for i in range(hp.Int("num_layers",1,10)):

    if counter == 0:

      model.add(Dense(hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"]),input_dim=8))
      model.add(Dropout(hp.Choice("dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])))
      model.add(Dropout(hp.Choice("dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1

  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=hp.Choice("optimizer",values=["rmsprop","adam","sgd","nadam"]),loss="binary_crossentropy",metrics=["accuracy"])
  return model



In [72]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=3,directory="mydir",project_name="final1")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 15s


In [74]:
tuner.get_best_hyperparameters()[0].values

{'units': 124,
 'num_layers': 6,
 'units0': 16,
 'activation0': 'sigmoid',
 'dropout0': 0.9,
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1}